In [5]:
#该程序计算量较大，最终的准确率达到0.97左右，可以不需要太多的中间隐藏层神经网络，从而减少计算量，同样可以达到较高的准确率，该程序仅作测试使用
#Dropout适用于使用大型的神经网络来训练较小的数据集

import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data 

#载入数据集,从网络中获取，使用one_hot来将标签变为0/1格式
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

#每个批次大小
batch_size = 50
#计算一共有多少批次  //为整除
n_batch = mnist.train.num_examples // batch_size

#定义两个占位符
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

#W = tf.Variable(tf.zeros([784,10]))  #权值
#b = tf.Variable(tf.zeros([10]))      #偏置

#第二种初始化方法
W1 = tf.Variable(tf.truncated_normal(([784,2000]),stddev=0.1)) #使用正态分布初始化，标准差为0.1
b1 = tf.Variable(tf.zeros([2000])+0.1)
L1 = tf.nn.tanh(tf.matmul(x,W1)+b1)  #激活函数选用双曲正切
L1_drop = tf.nn.dropout(L1,keep_prob)  #L1为某一层神经元的输出，参数keep_prob可以设计神经元工作的个数
#增加几个隐藏层
W2 = tf.Variable(tf.truncated_normal(([2000,2000]),stddev=0.1))#使用正态分布初始化，标准差为0.1
b2 = tf.Variable(tf.zeros([2000])+0.1)
L2 = tf.nn.tanh(tf.matmul(L1_drop,W2)+b2)  #激活函数选用双曲正切
L2_drop = tf.nn.dropout(L2,keep_prob)  #L1为某一层神经元的输出，参数keep_prob可以设计神经元工作的个数

W3 = tf.Variable(tf.truncated_normal(([2000,1000]),stddev=0.1)) #使用正态分布初始化，标准差为0.1
b3 = tf.Variable(tf.zeros([1000])+0.1)
L3 = tf.nn.tanh(tf.matmul(L2_drop,W3)+b3)  #激活函数选用双曲正切
L3_drop = tf.nn.dropout(L3,keep_prob)  #L1为某一层神经元的输出，参数keep_prob可以设计神经元工作的个数

W4 = tf.Variable(tf.truncated_normal(([1000,10]),stddev=0.1)) #使用正态分布初始化，标准差为0.1
b4 = tf.Variable(tf.zeros([10])+0.1)

prediction = tf.nn.softmax(tf.matmul(L3_drop,W4)+b4)  #将信号总和转换为概率值

#定义二次代价函数
#loss = tf.reduce_mean(tf.square(y-prediction))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#使用梯度下降法训练
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

#初始化变量                
init = tf.global_variables_initializer()
                
#比较两个参数是否相同，返回布尔型列表，用于检测训练效果,argmax函数用于求最大概率的位置（标签），返回以为张量中最大值所再位置
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))               
#求准确率,cast将对比的布尔型转换为浮点型，然后求平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
                
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(21): 
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)   #把训练集中的图片循环一次
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
                
        test_acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})  #使用测试集进行测试
        train_acc = sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0}) #使用训练集进行测试
        print("Iter" + str(epoch)+ ",Test Accuracy" + str(test_acc)+"Train Accuracy" + str(train_acc))       

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Iter0,Test Accuracy0.8455Train Accuracy0.8536364


KeyboardInterrupt: 